# Thư viện

In [1]:
import pandas as pd

import mysql.connector
from datetime import datetime, timedelta


# Kết nối đến MySQL

In [2]:
conn = mysql.connector.connect(host='10.90.21.39', port='3339', username='thuypt8', password='dmJ2sP9xTTU62sZq', database='um')
my_cursor = conn.cursor()
query = "SELECT * FROM loyalty_active_user LIMIT 2000000"
my_cursor.execute(query)
rows = my_cursor.fetchall()
columns = [i[0] for i in my_cursor.description]
dataSQL = pd.DataFrame(rows, columns=columns)

In [3]:
conn.commit()
conn.close()

print("Connection succesfully created!")

Connection succesfully created!


In [4]:
df = dataSQL

print(df)

        log_month    log_date  max_block_number      max_block_time  \
0         2023-08  2023-08-15             33531 2023-08-15 23:59:59   
1         2023-08  2023-08-15             33531 2023-08-15 23:59:59   
2         2023-08  2023-08-15             33531 2023-08-15 23:59:59   
3         2023-08  2023-08-15             33531 2023-08-15 23:59:59   
4         2023-08  2023-08-15             33531 2023-08-15 23:59:59   
...           ...         ...               ...                 ...   
1999995   2024-01  2024-01-13           4797956 2024-01-13 23:59:56   
1999996   2024-01  2024-01-13           4797956 2024-01-13 23:59:56   
1999997   2024-01  2024-01-13           4797956 2024-01-13 23:59:56   
1999998   2024-01  2024-01-13           4797956 2024-01-13 23:59:56   
1999999   2024-01  2024-01-13           4797956 2024-01-13 23:59:56   

                      loyalty_uid is_ggid  earn_1  earn_2  earn_3  burn_1  \
0        0658_2399887766019178496     UID       0       0    1000     

# Tính theo log_month

In [5]:
df1 = df.groupby(['loyalty_uid','log_month']).size().reset_index()

df1 = df1.rename(columns={'log_month': 'login_month'}).drop(0, axis=1)
print(df1)

                     loyalty_uid login_month
0       0122_1028121515778262387     2023-11
1       0122_1028121515778262387     2023-12
2        0122_106134295963268942     2023-11
3        0122_106134295963268942     2023-12
4       0122_1069468179007035104     2023-11
...                          ...         ...
869403          3f52d0af8245e981     2023-08
869404          3f5385492785e981     2023-08
869405          3f7639b24f6fb981     2023-10
869406          3f7725df5278e981     2023-08
869407          3f7725df5278e981     2023-10

[869408 rows x 2 columns]


In [6]:
df2 = df.groupby('loyalty_uid').agg({
    'log_month': 'min'
})
df2 = df2.rename(columns={'log_month': 'first_month'})
print(df2)

                         first_month
loyalty_uid                         
0122_1028121515778262387     2023-11
0122_106134295963268942      2023-11
0122_1069468179007035104     2023-11
0122_1070033982756837614     2023-11
0122_1076506494651560391     2023-11
...                              ...
3f442d7ba7c6f981             2023-08
3f52d0af8245e981             2023-08
3f5385492785e981             2023-08
3f7639b24f6fb981             2023-10
3f7725df5278e981             2023-08

[556798 rows x 1 columns]


In [7]:
merged_df = pd.merge(df1, df2, how='inner',on='loyalty_uid')

print(merged_df)

                     loyalty_uid login_month first_month
0       0122_1028121515778262387     2023-11     2023-11
1       0122_1028121515778262387     2023-12     2023-11
2        0122_106134295963268942     2023-11     2023-11
3        0122_106134295963268942     2023-12     2023-11
4       0122_1069468179007035104     2023-11     2023-11
...                          ...         ...         ...
869403          3f52d0af8245e981     2023-08     2023-08
869404          3f5385492785e981     2023-08     2023-08
869405          3f7639b24f6fb981     2023-10     2023-10
869406          3f7725df5278e981     2023-08     2023-08
869407          3f7725df5278e981     2023-10     2023-08

[869408 rows x 3 columns]


In [8]:
new_df = merged_df

new_df['login_month'] = pd.to_datetime(new_df['login_month'])
new_df['login_month'] = new_df['login_month'].dt.month   



new_df['first_month'] = pd.to_datetime(new_df['first_month'])
new_df['first_month'] = new_df['first_month'].dt.month
         
new_df['month_number'] = new_df['login_month'] - new_df['first_month']
print(new_df)

                     loyalty_uid  login_month  first_month  month_number
0       0122_1028121515778262387           11           11             0
1       0122_1028121515778262387           12           11             1
2        0122_106134295963268942           11           11             0
3        0122_106134295963268942           12           11             1
4       0122_1069468179007035104           11           11             0
...                          ...          ...          ...           ...
869403          3f52d0af8245e981            8            8             0
869404          3f5385492785e981            8            8             0
869405          3f7639b24f6fb981           10           10             0
869406          3f7725df5278e981            8            8             0
869407          3f7725df5278e981           10            8             2

[869408 rows x 4 columns]


In [9]:

def calculate_retention_rate(df):
    retention_rate = {}
    for month_number in range(1, df['month_number'].max() + 1):
        current_month = set(df[df['month_number'] == month_number]['loyalty_uid'])
        first_month_users = set(df[df['month_number'] == 0]['loyalty_uid'])
        retention_rate[month_number] = len(current_month.intersection(first_month_users)) / len(first_month_users) * 100
    return retention_rate

retention_rate_result = calculate_retention_rate(new_df)

# In ra kết quả
for month_number, rate in retention_rate_result.items():
    print(f'Retention Rate for Month {month_number}: {rate}')


Retention Rate for Month 1: 29.081282619549636
Retention Rate for Month 2: 12.251480788364901
Retention Rate for Month 3: 4.136149914331589
Retention Rate for Month 4: 1.1616421035995101


# Tính theo log_date

In [10]:
new_df1 = df.groupby(['loyalty_uid','log_date']).size().reset_index()

new_df1 = new_df1.rename(columns={'log_date': 'login_date'}).drop(0, axis=1)
print(new_df1)

new_df2 = df.groupby('loyalty_uid').agg({
    'log_date': 'min'
})
new_df2 = new_df2.rename(columns={'log_date': 'first_date'})
print(new_df2)

                      loyalty_uid  login_date
0        0122_1028121515778262387  2023-11-20
1        0122_1028121515778262387  2023-12-28
2         0122_106134295963268942  2023-11-20
3         0122_106134295963268942  2023-12-28
4        0122_1069468179007035104  2023-11-20
...                           ...         ...
1999995          3f5385492785e981  2023-08-15
1999996          3f7639b24f6fb981  2023-10-11
1999997          3f7725df5278e981  2023-08-30
1999998          3f7725df5278e981  2023-08-31
1999999          3f7725df5278e981  2023-10-27

[2000000 rows x 2 columns]
                          first_date
loyalty_uid                         
0122_1028121515778262387  2023-11-20
0122_106134295963268942   2023-11-20
0122_1069468179007035104  2023-11-20
0122_1070033982756837614  2023-11-20
0122_1076506494651560391  2023-11-20
...                              ...
3f442d7ba7c6f981          2023-08-29
3f52d0af8245e981          2023-08-29
3f5385492785e981          2023-08-15
3f7639b24f6fb

In [11]:
new_merged_df = pd.merge(new_df2, new_df1, how='inner',on='loyalty_uid')

print(new_merged_df)

                      loyalty_uid  first_date  login_date
0        0122_1028121515778262387  2023-11-20  2023-11-20
1        0122_1028121515778262387  2023-11-20  2023-12-28
2         0122_106134295963268942  2023-11-20  2023-11-20
3         0122_106134295963268942  2023-11-20  2023-12-28
4        0122_1069468179007035104  2023-11-20  2023-11-20
...                           ...         ...         ...
1999995          3f5385492785e981  2023-08-15  2023-08-15
1999996          3f7639b24f6fb981  2023-10-11  2023-10-11
1999997          3f7725df5278e981  2023-08-30  2023-08-30
1999998          3f7725df5278e981  2023-08-30  2023-08-31
1999999          3f7725df5278e981  2023-08-30  2023-10-27

[2000000 rows x 3 columns]


In [12]:

new_merged_df['login_date'] = pd.to_datetime(new_merged_df['login_date'])
new_merged_df['first_date'] = pd.to_datetime(new_merged_df['first_date'])
new_merged_df['day_number'] = (new_merged_df['login_date'] - new_merged_df['first_date'])

print(new_merged_df)

                      loyalty_uid first_date login_date day_number
0        0122_1028121515778262387 2023-11-20 2023-11-20     0 days
1        0122_1028121515778262387 2023-11-20 2023-12-28    38 days
2         0122_106134295963268942 2023-11-20 2023-11-20     0 days
3         0122_106134295963268942 2023-11-20 2023-12-28    38 days
4        0122_1069468179007035104 2023-11-20 2023-11-20     0 days
...                           ...        ...        ...        ...
1999995          3f5385492785e981 2023-08-15 2023-08-15     0 days
1999996          3f7639b24f6fb981 2023-10-11 2023-10-11     0 days
1999997          3f7725df5278e981 2023-08-30 2023-08-30     0 days
1999998          3f7725df5278e981 2023-08-30 2023-08-31     1 days
1999999          3f7725df5278e981 2023-08-30 2023-10-27    58 days

[2000000 rows x 4 columns]


In [13]:
data_day = new_merged_df.groupby(['first_date', 'day_number']).size()
data_day = data_day.reset_index()
print(data_day)

      first_date day_number     0
0     2023-08-14     0 days  3503
1     2023-08-14     1 days  1269
2     2023-08-14     2 days  1005
3     2023-08-14     3 days   916
4     2023-08-14     4 days   885
...          ...        ...   ...
11836 2024-01-12     1 days    71
11837 2024-01-12     2 days    17
11838 2024-01-13     0 days  1401
11839 2024-01-13     1 days    74
11840 2024-01-14     0 days   457

[11841 rows x 3 columns]


In [14]:
# lấy data với first date là 20-10-2023

data = new_merged_df[new_merged_df['first_date'] == '2023-10-20']
print(data)
data.to_clipboard()

                      loyalty_uid first_date login_date day_number
18612    0221_1211463329378246656 2023-10-20 2023-10-20     0 days
18811    0221_1220423089611907072 2023-10-20 2023-10-20     0 days
18812    0221_1220423089611907072 2023-10-20 2023-10-30    10 days
21402    0221_1428580904598544384 2023-10-20 2023-10-20     0 days
21403    0221_1428580904598544384 2023-10-20 2023-11-02    13 days
...                           ...        ...        ...        ...
1957332          3e7eda2f87f52981 2023-10-20 2023-11-30    41 days
1957333          3e7eda2f87f52981 2023-10-20 2023-12-12    53 days
1957334          3e7eda2f87f52981 2023-10-20 2023-12-25    66 days
1957335          3e7eda2f87f52981 2023-10-20 2024-01-01    73 days
1986316          3ec06e93b206d981 2023-10-20 2023-10-20     0 days

[2934 rows x 4 columns]


In [15]:
data_day = new_merged_df.groupby('day_number').size()

print(data_day)

day_number
0 days      556798
1 days       71971
2 days       58717
3 days       47268
4 days       40950
             ...  
148 days        44
149 days        62
150 days        34
151 days        15
152 days         5
Length: 153, dtype: int64


In [16]:
def calculate_retention_rate_date(df, datetime_str):
    datetime_value = pd.to_datetime(datetime_str)
    new_merged_df['login_date'] = pd.to_datetime(new_merged_df['login_date'])
    new_merged_df['first_date'] = pd.to_datetime(new_merged_df['first_date'])
    retention_rate = {}
    for date_number in range(0, 31):
        end_of_date = datetime_value + timedelta(days=date_number)
        # print(datetime_value, end_of_month)

        new_users = set(df[(df['login_date'] == end_of_date)]['loyalty_uid'])

        first_date_users = set(df[df['first_date'] == datetime_value]['loyalty_uid'])

        retention_rate[date_number] = len(new_users.intersection(first_date_users)) / len(first_date_users) * 100
    return retention_rate




In [17]:
date_input = input('Nhập ngày cần tính: ') # 2023-10-20
retention_rate_result1 = calculate_retention_rate_date(new_merged_df, date_input)

# In ra kết quả
print(f'Date: {date_input}')
for month_number, rate in retention_rate_result1.items():
    print(f'Retention Rate for day {month_number}: {rate}')

Date: 2023-10-11
Retention Rate for day 0: 100.0
Retention Rate for day 1: 3.3131457071606696
Retention Rate for day 2: 2.1375133594584965
Retention Rate for day 3: 4.061275382971143
Retention Rate for day 4: 1.9237620235126471
Retention Rate for day 5: 1.2468827930174564
Retention Rate for day 6: 0.46312789454934095
Retention Rate for day 7: 0.8193801211257571
Retention Rate for day 8: 3.0281439258995366
Retention Rate for day 9: 1.7100106875667973
Retention Rate for day 10: 1.2825080156750979
Retention Rate for day 11: 0.8193801211257571
Retention Rate for day 12: 1.5318845742785894
Retention Rate for day 13: 1.4962593516209477
Retention Rate for day 14: 1.2112575703598147
Retention Rate for day 15: 1.816886355539722
Retention Rate for day 16: 1.1756323477021733
Retention Rate for day 17: 0.7481296758104738
Retention Rate for day 18: 1.0687566797292483
Retention Rate for day 19: 1.3181332383327395
Retention Rate for day 20: 0.7837548984681154
Retention Rate for day 21: 0.676879230495